In [26]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
from sklearn import preprocessing
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [27]:
most_important_churn = ['eqpdays','hnd_price','hnd_webcap','totmrc_Mean','asl_flag','crclscod','mou_Mean','mou_cvce_Mean',
 'complete_Mean','comp_vce_Mean','mou_opkv_Mean','avg3mou','attempt_Mean','plcd_vce_Mean','peak_vce_Mean','opk_vce_Mean',
 'mou_peav_Mean','mou_rvce_Mean','models','avg3qty','owylis_vce_Mean','area','avg6mou','recv_vce_Mean','ethnic','phones','uniqsubs',
 'iwylis_vce_Mean','lor','avg6qty','unan_vce_Mean','ccrndmou_Mean','mouowylisv_Mean','custcare_Mean','inonemin_Mean',
 'cc_mou_Mean','mouiwylisv_Mean','callwait_Mean','refurb_new','dualband','change_mou','threeway_Mean','avgmou', 'churn']

most_important = ['eqpdays','hnd_price','hnd_webcap','totmrc_Mean','asl_flag','crclscod','mou_Mean','mou_cvce_Mean',
 'complete_Mean','comp_vce_Mean','mou_opkv_Mean','avg3mou','attempt_Mean','plcd_vce_Mean','peak_vce_Mean','opk_vce_Mean',
 'mou_peav_Mean','mou_rvce_Mean','models','avg3qty','owylis_vce_Mean','area','avg6mou','recv_vce_Mean','ethnic','phones','uniqsubs',
 'iwylis_vce_Mean','lor','avg6qty','unan_vce_Mean','ccrndmou_Mean','mouowylisv_Mean','custcare_Mean','inonemin_Mean',
 'cc_mou_Mean','mouiwylisv_Mean','callwait_Mean','refurb_new','dualband','change_mou','threeway_Mean','avgmou']

In [29]:

dataframe = pd.read_csv('train.csv').drop(['Customer_ID'], axis=1)

dataframe = dataframe.loc[:, most_important_churn]


quantile_value = 0.999


CATEGORICAL_COLUMNS = list(dataframe.select_dtypes(include='object'))
NUMERIC_COLUMNS = list(dataframe.select_dtypes(include=['float64', 'int64']))


for column in NUMERIC_COLUMNS:
    dataframe[column] = dataframe[column].fillna(dataframe[column].median())
    q_high = dataframe[column].quantile(quantile_value)
    q_low = dataframe[column].quantile(1 - quantile_value)
    dataframe = dataframe[(dataframe[column] <= q_high) & (dataframe[column] >= q_low)]
    dataframe[column]=((dataframe[column]-dataframe[column].min())/(dataframe[column].max()-dataframe[column].min()))
    
    #dataframe[column] = (dataframe[column] - dataframe[column].mean())/(dataframe[column].std())
    
for column in CATEGORICAL_COLUMNS:
    dataframe[column] = dataframe[column].fillna(dataframe[column].describe().top)

train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  print(dataframe.head()) 
  labels = dataframe.pop('churn')
  print(dataframe.head())  
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

46240 train examples
11560 validation examples
14450 test examples


In [3]:


batch_size = 25 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)




       rev_Mean  mou_Mean  totmrc_Mean   da_Mean  ovrmou_Mean  ovrrev_Mean  \
8840   0.185684  0.191475     0.339200  0.044693     0.000000     0.000000   
50905  0.068035  0.131391     0.116784  0.016760     0.007958     0.008516   
52275  0.041029  0.004064     0.097444  0.000000     0.000000     0.000000   
43163  0.068730  0.056713     0.161912  0.005587     0.000000     0.000000   
55801  0.154950  0.362998     0.323083  0.011173     0.038384     0.047925   

       vceovr_Mean  datovr_Mean  roam_Mean  change_mou  ...  forgntvl  ethnic  \
8840      0.000000          0.0        0.0    0.475765  ...       0.0       N   
50905     0.009389          0.0        0.0    0.515049  ...       0.0       I   
52275     0.000000          0.0        0.0    0.514686  ...       0.0       U   
43163     0.000000          0.0        0.0    0.526103  ...       0.0       S   
55801     0.052837          0.0        0.0    0.576129  ...       0.0       N   

       kid0_2  kid3_5  kid6_10  kid11_15  ki

In [30]:
feature_columns = []

NUMERIC_COLUMNS = NUMERIC_COLUMNS[:-1]
print(NUMERIC_COLUMNS)

# numeric cols
for header in NUMERIC_COLUMNS:
    feature_columns.append(feature_column.numeric_column(header))



#cat_col_tf = []

for col in CATEGORICAL_COLUMNS:
    thal = feature_column.categorical_column_with_vocabulary_list(
      col, dataframe[col].unique())
    thal_one_hot = feature_column.indicator_column(thal)
    feature_columns.append(thal_one_hot)
    
for col in CATEGORICAL_COLUMNS:
  vocabulary = dataframe[col].unique()
  cat_c = tf.feature_column.categorical_column_with_vocabulary_list(col, vocabulary)
  embeding = feature_column.embedding_column(cat_c, dimension=50)
  feature_columns.append(embeding)    

# embedding cols
#thal_embedding = feature_column.embedding_column(thal, dimension=8)
#feature_columns.append(thal_embedding)

# embedding cols
#for header in CATEGORICAL_COLUMNS:
 #   thal_embedding = feature_column.embedding_column(header, dimension=8)
  #  feature_columns.append(thal_embedding)



['eqpdays', 'hnd_price', 'totmrc_Mean', 'mou_Mean', 'mou_cvce_Mean', 'complete_Mean', 'comp_vce_Mean', 'mou_opkv_Mean', 'avg3mou', 'attempt_Mean', 'plcd_vce_Mean', 'peak_vce_Mean', 'opk_vce_Mean', 'mou_peav_Mean', 'mou_rvce_Mean', 'models', 'avg3qty', 'owylis_vce_Mean', 'avg6mou', 'recv_vce_Mean', 'phones', 'uniqsubs', 'iwylis_vce_Mean', 'lor', 'avg6qty', 'unan_vce_Mean', 'ccrndmou_Mean', 'mouowylisv_Mean', 'custcare_Mean', 'inonemin_Mean', 'cc_mou_Mean', 'mouiwylisv_Mean', 'callwait_Mean', 'change_mou', 'threeway_Mean', 'avgmou']


In [31]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [32]:
batch_size = 100
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

        eqpdays  hnd_price hnd_webcap  totmrc_Mean asl_flag crclscod  \
70087  0.167404   0.128205       WCMB     0.282620        N       AA   
61324  0.037903   0.435897       WCMB     0.154184        N       AA   
28868  0.186987   0.128205       WCMB     0.050971        N       AA   
17095  0.143399   0.358974       WCMB     0.411106        Y       CA   
58562  0.396083   0.051282       WCMB     0.154133        N       AA   

       mou_Mean  mou_cvce_Mean  complete_Mean  comp_vce_Mean  ...  \
70087  0.144615       0.155431       0.138794       0.158854  ...   
61324  0.052771       0.018081       0.016686       0.014757  ...   
28868  0.021458       0.013843       0.026166       0.029948  ...   
17095  0.174949       0.119047       0.058020       0.063368  ...   
58562  0.105752       0.046919       0.139173       0.159288  ...   

       inonemin_Mean  cc_mou_Mean  mouiwylisv_Mean  callwait_Mean  refurb_new  \
70087       0.025303     0.000000          0.00000       0.058140      

In [36]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(256, activation='relu'),
  layers.BatchNormalization(), 
  layers.Dropout(0.5),
  layers.Dense(128, activation='relu'),
  layers.BatchNormalization(), 
  layers.Dropout(0.5),
  layers.Dense(64, activation='relu'),
  layers.BatchNormalization(), 
  layers.Dropout(0.5),
  #layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='relu'),
  #layers.Dropout(0.3),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

cal_back = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

model.fit(train_ds,
          validation_data=val_ds,
          epochs = 100)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/100
463/463 [==============================] - 35s 75ms/step - loss: 0.7941 - accuracy: 0.5158 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/100
463/463 [==============================] - 32s 70ms/step - loss: 0.6931 - accuracy: 0.5345 - val_loss: 0.6833 - val_accuracy: 0.5603
Epoch 3/100
463/463 [==============================] - 29s 64ms/step - loss: 0.6858 - accuracy: 0.5522 - val_loss: 0.6803 - val_accuracy: 0.5676
Epoch 4/100
463/463 [==============================] - 32s 68ms/s

KeyboardInterrupt: 

In [20]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

149/149 [==============================] - 3s 17ms/step - loss: 0.7618 - accuracy: 0.5816
Accuracy 0.58162993


In [22]:
datatest = pd.read_csv('test.csv')
X_test = datatest.drop(['Customer_ID'], axis=1)
#X_test = X_test.loc[:, most_important]


for column in NUMERIC_COLUMNS:
    X_test[column] = X_test[column].fillna(X_test[column].median())
    X_test[column]=((X_test[column]-X_test[column].min())/(X_test[column].max()-X_test[column].min()))
    
for column in CATEGORICAL_COLUMNS:
    X_test[column] = X_test[column].fillna(X_test[column].describe().top)

testing = tf.data.Dataset.from_tensor_slices((dict(X_test))).batch(batch_size=200)
preds = model.predict(testing)





In [23]:
preds = preds[:,0]

In [24]:
def save_result(probs):
    data_test = pd.read_csv('test.csv')
    pd_result = pd.DataFrame({"Customer_ID": data_test.Customer_ID, "churn": probs})
    pd_result.Customer_ID = pd_result.Customer_ID.astype(int)
    pd_result.to_csv('result.csv', index=False)

In [25]:
save_result(preds)